<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Fast_faiss_semantic_wikipediasearch_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers
!pip install wikipedia
!pip install faiss-cpu
!pip install nltk

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=c2d5081a4a47cf916cfbe86e2e1de23f054ef977fe35b3cb0d20b3515f1d6ab1
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
ERROR: unknown command "8nstall" - maybe you meant "install"


In [11]:
import requests
from sentence_transformers import SentenceTransformer, util
import faiss
import wikipedia
import hashlib
import os

# Configuration
CACHE_DIR = "wikipedia_cache"  # Directory to store cached Wikipedia content
EMBEDDING_MODEL = "all-mpnet-base-v2"  # Sentence transformer model
CHUNK_SIZE = 500  # Characters per chunk for long Wikipedia articles

# Create cache directory if it doesn't exist
os.makedirs(CACHE_DIR, exist_ok=True)

# Initialize sentence transformer model
model = SentenceTransformer(EMBEDDING_MODEL)

def fetch_and_cache_wikipedia(query):
    """Fetches Wikipedia content and caches it based on the query hash."""
    query_hash = hashlib.md5(query.encode()).hexdigest()
    cache_path = os.path.join(CACHE_DIR, f"{query_hash}.txt")

    if os.path.exists(cache_path):
        with open(cache_path, "r", encoding="utf-8") as f:
            content = f.read()
            print("Loaded from cache")
            return content

    try:
        page = wikipedia.page(query)  # Fetch Wikipedia page
        content = page.content
        with open(cache_path, "w", encoding="utf-8") as f:
            f.write(content)
        print("Fetched from Wikipedia and cached")
        return content

    except wikipedia.exceptions.PageError:
        print("Page not found on Wikipedia.")
        return None
    except wikipedia.exceptions.DisambiguationError as e:
        print("Disambiguation error:", e.options) # Handle disambiguation
        return None


def chunk_text(text, chunk_size):
    """Chunks text into smaller pieces for efficient processing."""
    return [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]



def search_wikipedia(query):
    """Searches Wikipedia for a given query and returns semantically similar sentences."""
    wikipedia_content = fetch_and_cache_wikipedia(query)

    if wikipedia_content is None:
        return [] # Return empty list if no content found

    sentences = wikipedia_content.split(". ")  # Simple sentence splitting

    # Chunking for very long articles
    chunked_sentences = []
    for sentence in sentences:
        chunked_sentences.extend(chunk_text(sentence, CHUNK_SIZE))
    sentences = chunked_sentences


    sentence_embeddings = model.encode(sentences)


    # Build FAISS index
    index = faiss.IndexFlatL2(sentence_embeddings.shape[1])  # Use L2 distance
    index.add(sentence_embeddings)

    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, 25)  # Search for top 5 similar sentences

    results = []
    for i in I[0]:
        results.append(sentences[i])
    return results



# Example usage:
query = "Amitabh  Bachan films"  # User query
similar_sentences = search_wikipedia(query)


if similar_sentences:
    print("\nTop similar sentences from Wikipedia:")
    for sentence in similar_sentences:
        print(sentence)

Fetched from Wikipedia and cached

Top similar sentences from Wikipedia:
Amitabh Bachchan (pronounced [əmɪˈt̪ɑːbʱ ˈbətːʃən] ; born Amitabh Srivastava; 11 October 1942) is an Indian actor who works in Hindi cinema
Dudes in the 10th Century (2003)
Parineeta (2005)
March of the Penguins (2005), Indian version
Swami (2007)
Jodhaa Akbar (2008)
Zor Lagaa Ke...Haiya! (2009)
Ra.One (2011)
Kahaani (2012)
Krrish 3 (2013)
Mahabharat (2013)
Kochadaiiyaan (Hindi Version) (2014)
CBI documentary (2014) – sanctioned by Central Bureau of Investigation
The Ghazi Attack (2017)
Firangi (2017)


=== Business investments ===
Around 1994, Bachchan started Amitabh Bachchan Corporation Ltd (ABCL), an event management, production
Bachchan starred as a young poet, Amit Malhotra, who falls deeply in love with a beautiful young girl named Pooja (Rakhee Gulzar) who ends up marrying someone else (Shashi Kapoor)
Except for the much delayed Insaniyat (1994) alongside Sunny Deol, which was highly anticipated and receiv